<div style="direction:rtl;line-height:300%;"><font face="XB Zar" size=5>
<div style="direction:rtl;line-height:300%;"><font face="XB Zar" size=5>
<div align=center>
<font face="IranNastaliq" size=30>
<p></p>
به نام خدا
</font>
<br>
<font color=#FF7500>
دانشگاه صنعتی شریف - دانشکده مهندسی کامپیوتر
</font>
<br>
<font color=blue>
داده‌ساختارها و الگوریتم‌ها
</font>
<br>
ترم دوم سال تحصیلی ۱۳۹۸-۱۳۹۷
</div>
<hr/>
<font color=red size=6>
<p></p>

<div align=center>فصل دهم، بخش چهارم: یافتن کوتاه‌ترین مسیر در گراف وزن‌دار</div>
</font>

<style type="text/css" scoped>
p{
border: 1px solid #a2a9b1;background-color: #f8f9fa;display: inline-block;
};
</style>

<div>
<h3> فهرست محتویات</h3>
<ul style="margin-right: 0;">
<li><a href="#section1"> تعریف مسئله</a></li>
<li><a href="#section1_5"> تعریف relaxation در گراف</a></li>
<li><a href="#section2">الگوریتم Dijkstra</a></li>
<li><a href="#section3">الگوریتم Bellman-Ford</a></li>
</ul>
</div>

<div style="direction:rtl;line-height:300%;"><font face="XB Zar" size=5>
<font color=#FF7500 id="section1">
مسئله یافتن کوتاه‌ترین مسیر در گراف
</font><p></p>
گراف وزن‌دار و جهت‌دار $G(V, E)$
داده شده که وزن‌های گراف از نگاشت مقادیر تابع
$w : E \rightarrow \mathbb{R}$
به یال‌های گراف بدست می‌آیند.
به عنوان مثال گراف زیر نمونه‌ای از گراف مورد‌نظر می‌باشد: 
<p></p>
<br>
<img src="image1.png" style="float:center; width:50%">
<br>
<p></p>
وزن یک میسر دلخواه مانند
$p$
در گراف که شامل راس های 
$p = <v_0, v_1, \cdots, v_k>$
می‌باشد به صورت جمع وزن‌هایی که در مسیر می‌باشند تعریف می‌شود:
$w(p) = \sum_{i=1}^{k}w(v_{i-1}, v_i)$
<br>
به عنوان مثال وزن مسیر 
$p = <3,6,2,4,5>$
در گراف بالا برابر با ۲ می‌باشد.
<br>
حال کوتاه‌ترین مسیر از راس
$u$
به راس
$v$
در گراف برابر است با مسیری از $u$ به $v$ که در بین تمام مسیر های ممکن بین این دو راس، کمترین وزن را داشته باشد
که وزن آن را با نماد 
$\delta (u,v)$
نمایش می‌دهیم.
به عنوان مثال کوتاه‌ترین مسیر از راس ۶ به راس ۱ در گراف بالا، مسیر 
$<6, 2, 4, 1>$
می‌باشد که وزن آن برابر با 
$\delta (6,1) = 3$
است.
<p></p>
در این دفترچه قصد داریم الگوریتم‌هایی برای پیدا کردن کوتاه ترین مسیر از یک راس مبدا دلخواه مانند 
$s$
به سایر رئوس گراف بیان کنیم که با عنوان
$single-source\;shortest-paths\;problem$
شناخته می‌شود.
<p></p>
</font></div>


In [49]:
class Node:
    def __init__(self, num):
        self.num = num
        self.d = 0 # will explain later !
        
class Graph:
    def __init__(self, nodes):
        self.V = nodes
        # assume that weight of edges are non zero
        # adjacency matrix representation
        self.E = [[0 for _ in range(len(nodes))] for _ in range(len(nodes))]
        
    def addEdge(self, u, v, w):
        self.E[u][v] = w

<div style="direction:rtl;line-height:300%;"><font face="XB Zar" size=5>
<font color=#FF7500 id="section1_5">
$Relaxation$
</font><p></p>
الگوریتم‌هایی که در ادامه بیان خواهیم کرد از تکنیک
$Relax$
کردن یال‌ها طی اجرای الگوریتم استفاده می‌کنند و در واقع تفاوتشان در
تعداد استفاده از آن است.
در ابتدا برچسب‌ای بر روی هر راس 
$v \in V$
می‌گذاریم که نشان‌دهنده تخمین کوتاه ترین فاصله از راس مبدا($s$) به آن راس می‌باشد و آن را با
$v.d$
نشان می‌دهیم
که مقدار ابتدایی آن برای هر راس  
$v \in V-\{s\}$
برابر با
$\infty$
و برای $s$ مقدار آن را صفر قرار می‌دهیم.
</font></div>

In [84]:
import math
def initialize_single_source(G, s):
    for v in G.V:
        v.d = math.inf
    s.d = 0

<div style="direction:rtl;line-height:300%;"><font face="XB Zar" size=5>
<p></p>
حال در هر مرحله از اجرای الگوریتم، 
$relax$
کردن یال 
$(u,v)$
بدین معناست که چک کنیم آیا می‌توانیم تخمین کوتاه‌ترین فاصله به
$v$
را با گذر‌کردن از راس $u$ و در نتیجه یال
$(u,v)$
بهتر کنیم یا نه که اگر توانستیم مقدار
$v.d$
را
$update$
می‌کنیم.
در هر مرحله از 
$relax$
کردن یک یال ممکن است مقدار تخمین کوتاه‌ترین فاصله به 
$v$($v.d$)
کاهش یابد و یا تغییری نکند.
<img src="image2.png" style="float:center; width:50%">
</font></div>

In [85]:
def relax(G, u, v):
    if v.d > u.d + G.E[u.num][v.num]:
        v.d = u.d + G.E[u.num][v.num]

<div style="direction:rtl;line-height:300%;"><font face="XB Zar" size=5>
<font color=#FF7500 id="section2">
الگوریتم Dijkstra
</font><p></p>
برای اجرای الگوریتم، مجموعه
$S$
را برابر با راس‌هایی در نظر می‌گیریم که وزن کوتاه‌ترین مسیر از 
$s$
به آن‌ها را یافته‌ایم. این مجموعه در ابتدا تهی می‌باشد.
حال در هر مرحله راس $u$ از راس‌های
$u \in V-S$
(راس‌هایی که در $S$ نیستند)
را که دارای کمترین
مقدار 
$u.d$
می‌باشد را به
$S$
اضافه می‌کنیم سپس تمام یال‌هایی را که به
$u$
وصل می‌باشند را 
$relax$
می‌کنیم.
این کار را تا زمانی که تمام راس‌های گراف به مجموعه 
$S$
اضافه نشده‌اند، انجام می‌دهیم.
حال ادعا می‌کنیم که تمامی رئوس که در انتها در $S$ می‌باشند،
$v.d$ شان برابر با کوتاه‌ترین مسیر ممکن از $s$ به آن‌ها است.
<p></p>
مراحل اجرای الگوریتم روی یک گراف به شکل زیر می‌باشد:
<img src="image3.png" style="float:center; width:80%">
توضیح شکل: در ابتدا از بین رئوس گراف، راس 
$s$
به مجموعه $S$ اضافه می‌شود زیرا مقدار $s.d$ برابر با صفر می‌باشد در صورتی سایر راس‌ها مقدار $\infty$ دارند.
سپس با $relax$ کردن یال‌های متصل به $s$، $t.d = 10$ و $y.d = 5$ می‌شود.
حال از بین رئوس خارج از $S$، راس $y$ که دارای کمترین تخمین است به $S$ اضافه می‌شود و یال های متصل به آن $relax$ می‌شوند.
اضافه کردن رئوس و $relax$ کردن یال‌های آن تا زمانی که $S \neq V$ است، ادامه پیدا می‌کند.
<p></p>
اثبات درستی الگوریتم:
<br>
ادعا می‌کنیم برای هر راس $v$ای که به مجموعه $S$
اضافه می‌شود داریم
$v.d = \delta(s,v)$
و همچنین کوتاه‌ترین مسیر از $s$ به $v$ در مجموعه $S$ قرار دارد.
<br>
برای اثبات این ادعا از فرض خلف استفاده می‌کنیم بدین منظور راس $u$ را اولین راسی بگیرید که با ویژگی $u.d > \delta(s,u)$ به مجموعه $S$ اضافه می‌شود.
در نتیجه کوتاه‌ترین مسیر از $s$ به $u$ در مجموعه S قرار ندارد،
این مسیر ($P$)را برابر با 
$s \rightsquigarrow x \rightarrow y \rightsquigarrow u$ در نظر می‌گیریم.
(بخشی از این مسیر حتما در $S$ قرار دارد زیرا راس$s$ در این مجموعه می‌باشد.)
<img src="image4.png" style="float:center; width:40%">
<p></p>
$y$ اولین راسی است در مسیر $P$ که در $S$ قرار ندارد و راس  $x$ را راسی می‌گیریم که درست قبل از $y$ روی مسیر $P$، در $S$ قرار داشته باشد.
طبق اجرای الگوریتم می‌دانیم:
$u.d \leq y.d$
زیرا در هر مرحله الگوریتم راسی را انتخاب می‌کند که دارای کمترین $d$ باشد و در این مرحله راس $u$ انتخاب شده است پس این راس بین راس هایی که در $S$ قرار ندارند، دارای کمترین مقدار $d$ می‌باشد.
پس خواهیم داشت :
<br>
\begin{align*}
& u.d \leq y.d \leq x.d + w(x,y) \qquad \qquad \qquad \quad (by \; relaxation\; of\; (x,y))\\
& \qquad \qquad = \delta(s,x) + w(x,y) \qquad \qquad \qquad \quad (by \; hypo.)\\
& \qquad \qquad = \delta(s,y) \leq \delta(s,y) + w(P_2)\\
& \qquad \qquad \qquad \quad \; = \delta(s,u)\\
& \implies u.d \leq \delta(s,u)\\
\end{align*}
که با فرض $u.d > \delta(s,u)$ در تناقض است، پس حکم ثابت شد.
<br>
خط سوم با فرض اینکه وزن یال‌ها بزرگتر مساوی با صفر می‌باشد، درست است در نتیجه الگوریتم $Dijkstra$
فقط برای گراف‌هایی که وزن منفی ندارند درست کار می‌کند.
<br>
به عنوان مثال قطعه کد زیر را بر روی گرافی که در ابتدای دفترچه بیان کردیم که دارای یال با وزن منفی می‌باشد، اجرا کنید. 
</font></div>

In [86]:
def find_min_distance(G, S):
    minimum = math.inf
    for v in G.V:
        if v.d < minimum and S[v.num] == False:
            minimum = v.d
            min_node = v
            
    return min_node


def dijkstra(G, s):
    
    initialize_single_source(G, s)
    S = [False] * len(G.V)
    
    for _ in range(len(G.V)):
        u = find_min_distance(G, S)
        S[u.num] = True
        
        for v in G.V:
            if G.E[u.num][v.num] > 0 and S[v.num] == False:
                relax(G, u, v)
    
    for v in G.V:
        print("vertex num " + str(v.num) + "   " + "min distance from Source: " + str(v.d))

nodes = [Node(i) for i in range(5)]

g = Graph(nodes) # s = 0, t = 1, x = 2, z = 3, y = 4

g.addEdge(0, 1, 10)
g.addEdge(0, 4, 5)
g.addEdge(1, 2, 1)
g.addEdge(1, 4, 2)
g.addEdge(2, 3, 4)
g.addEdge(3, 2, 6)
g.addEdge(3, 0, 7)
g.addEdge(4, 1, 3)
g.addEdge(4, 2, 9)
g.addEdge(4, 3, 2)

dijkstra(g, g.V[0])

vertex num 0   min distance from Source: 0
vertex num 1   min distance from Source: 8
vertex num 2   min distance from Source: 9
vertex num 3   min distance from Source: 7
vertex num 4   min distance from Source: 5


<div style="direction:rtl;line-height:300%;"><font face="XB Zar" size=5>
مرتبه زمانی کد بالا از $O(|V|^2)$
 می‌باشد که می‌توان آن را با استفاده از داده‌ساختار $min-Heap$ برای بدست آوردن کمترین مقدار $d$،
 تا مرتبه زمانی $O(|E|log|V|)$
 بهبود بخشید که می‌توانید به عنوان تمرین در $pycharm\;edu$ مربوط به این دفترچه انجام دهید.
</font></div>

<div style="direction:rtl;line-height:300%;"><font face="XB Zar" size=5>
<font color=#FF7500 id="section3" size=5>
الگوریتم $Bellman-Ford$
</font><p></p>
دیدیم که الگوریتم $Dijkstra$ برای گراف‌هایی که دارای یال‌هایی با وزن منفی هستند درست کار نمی‌کرد، حال الگوریتمی که در ادامه خواهیم گفت برای این نوع گراف‌ها نیز به درستی کار می‌کند.
در ابتدا باید برای گراف داده شده این شرط را داشته باشیم که گراف دارای دور با وزن منفی نباشد زیرا اگر گراف دارای چنین دوری باشد، می‌توان با گذر‌کردن در این دور به هر تعداد دلخواهی وزن مسیر را کاهش داد.
در نتیجه مسئله دارای جواب نیست. در ادامه بیان می‌کنیم که می‌توان با استفاده از از الگوریتم $bellman-ford$ وجود چنین دوری را در گراف تشخیص داد.
<br>
حال فرض می‌کنیم گراف داده شده دارای دور با وزن منفی نباشد، الگوریتم به این صورت کار می‌کند که همه‌ی یال های گراف را$(|V|-1)$ بار $relax$ می‌کند. در این صورت در انتها برای هر راس $v \in V$ خواهیم داشت که 
$v.d = \delta(s, v)$ است.
<img src="image5.png" style="float:center; width:80%">
<br>
برای سادگی کد و بهتر شدن زمان اجرای الگوریتم از $adjacency\;list$ برای ذخیره‌سازی گراف استفاده می‌کنیم.
چون تمامی یال‌های گراف $|V|-1$ بار $relax$ می‌شوند، مرتبه زمانی الگوریتم از $O(|V||E|)$ است.
</font></div>

In [94]:
class graph:
    def __init__(self, vertices):
        self.V = vertices
        self.E = []
    
    def addEdge(self, u, v, w):
        self.E.append([self.V[u], self.V[v], w])

        
def bellman_ford(G, s):
    
    initialize_single_source(G, s)
    for _ in range(len(G.V) - 1):
        
        for u, v, w in G.E:
            
            if u.d + w < v.d: # relax
                v.d = u.d + w
                
    for v in G.V:
        print("vertex num " + str(v.num) + "   " + "min distance from Source: " + str(v.d))

nodes = [Node(i) for i in range(5)]
g = graph(nodes) # s = 0, t = 1, x = 2, z = 3, y = 4

g.addEdge(0, 1, 6)
g.addEdge(0, 4, 7)
g.addEdge(1, 2, 5)
g.addEdge(1, 3, -4)
g.addEdge(1, 4, 8)
g.addEdge(2, 1, -2)
g.addEdge(3, 2, 7)
g.addEdge(3, 0, 2)
g.addEdge(4, 3, 9)
g.addEdge(4, 2, -3)

bellman_ford(g, g.V[0])

vertex num 0   min distance from Source: 0
vertex num 1   min distance from Source: 2
vertex num 2   min distance from Source: 4
vertex num 3   min distance from Source: -2
vertex num 4   min distance from Source: 7


<div style="direction:rtl;line-height:300%;"><font face="XB Zar" size=5>
اثبات درستی الگوریتم:
<br>
می‌خواهیم نشان دهیم اگر گراف دور با وزن منفی نداشته باشد برای هر $v \in V$ داریم که $d_{n-1}(v) = \delta(s,v)$
(منظور از $d_{n-1}(v)$ مقدار $v.d$ در مرحله $n-1$ام در $relax$کردن یال‌ها است.)
<br>
حال با استقرا بر روی $k$ نشان می‌دهیم که $d_k(v)$ برابر است با کمترین وزن مسیر از $s$ به $v$ در صورتی که حداکثر از $k$ یال استفاده کند. با اثبات این حکم درستی الگوریتم نشان داده می‌شود زیرا هر مسیر ساده از $s$ به سایر رئوس حداکثر دارای $n-1$ یال می‌باشد.
<br>
پایه استقرا: اگر $k = 0$ باشد در این صورت با استفاده از صفر یال فقط به راس $s$ می‌رسیم که مقدار $d$ آن صفر می‌باشد و کمترین مقدار برای راس $s$ است در نتیجه حکم برقرار است.
<br>
حال فرض می‌کنیم برای هر راس $u \in V$، $d_{k-1}(u)$ کمترین وزن مسیر از $s$ به $u$ با استفاده از حداکثر $k-1$ یال 
است.
مسیر$P$ را کوتاه‌ترین مسیر به راس $v$ با حداکثر $k$ یال از $s$  در نظر بگیرید،
راس قبلی $v$ در مسیر $P$ را $u$ و مسیر $s$ به $u$ را $Q$ می‌نامیم.
حال چون مسیر $Q$ دارای حداکثر $k-1$ یال می‌باشد پس کوتاه‌ترین مسیر از $s$ به $u$ است زیرا در غیر این صورت با فرض کوتاه‌ترین مسیر بودن P به تناقض می‌خورد. در نتیجه طبق فرض استقرا $w(Q)$ برابر با $d_{k-1}(u)$ خواهد شد.
<br>
در مرحله $k$ ام از اجرای الگوریتم، مقدار $d_k(v)$ به $d_{k}(v) = min(d_{k-1}(v), d_{k-1}(u) + w(u,v))$، $update$ می‌شود که در آن $d_{k-1}(u) + w(u,v)$ برابر با $w(P)$ است. حال واضح است که $w(P) \leq d_{k-1}(v)$ می‌باشد
چون مسیر $P$ می‌تواند حداقل یک یال بیشتر از مسیر با وزن $d_{k-1}(v)$ داشته باشد پس مسیر $P$ یا همان مسیر می‌ماند یا با اضافه کردن یال‌هایی با وزن منفی از وزن مسیر کاسته می‌شود. در نتیجه  $d_k(v) = w(P)$
است و حکم استقرا ثابت می‌شود.
<br>
<br>
تشخیص دور با وزن منفی:
<br>
برای اینکه تشخیص دهیم گراف دارای دور با وزن منفی هست یا نه، کافی است ابتدا الگوریتم $bellman-ford$ را روی گراف اجرا کنیم و مقادیر $v.d$ را برای هر راس $v \in V$ یادداشت کنیم سپس یک بار تمامی یال‌های گراف را $relax$ کرده و مقادیر $v.d$ جدید را با قبلی مقایسه کنیم، اگر راسی وجود داشت که تغییر کرده باشد در آن صورت گراف دارای دور با وزن منفی می‌باشد. 
</font></div>

In [93]:
def detecting_negative_cycle(G):
    bellman_ford(G, G.V[0])
    first_list = []
    
    for v in G.V:
        first_list += [v.d]
        
    for u, v, w in G.E:
        if u.d + w < v.d: # relax
                v.d = u.d + w
    
    for i in range(len(G.V)):
        if G.V[i].d != first_list[i]:
            return True
    
    return False

nodes = [Node(i) for i in range(5)]
g = graph(nodes) # s = 0, t = 1, x = 2, z = 3, y = 4

g.addEdge(0, 1, 6)
g.addEdge(0, 4, 7)
g.addEdge(1, 2, 5)
g.addEdge(1, 3, -4)
g.addEdge(1, 4, 8)
g.addEdge(2, 1, -2)
g.addEdge(3, 2, 7)
g.addEdge(3, 0, 2)
g.addEdge(4, 3, 9)
g.addEdge(4, 2, -3)

detecting_negative_cycle(g)

vertex num 0   min distance from Source: 0
vertex num 1   min distance from Source: 2
vertex num 2   min distance from Source: 4
vertex num 3   min distance from Source: -2
vertex num 4   min distance from Source: 7


False